In [22]:
import requests
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [32]:
df = pd.read_csv("../data/clean/bitcoin_prices.csv")

percent_change = (df["Close"] - df["Close"].shift(1)) / df["Close"]
# plt.plot(np.asarray(df["Time"], dtype='datetime64[s]'), percent_change)

y = np.zeros(percent_change.shape)
y[np.where(percent_change > 0)] = 1

def moving_average(x, w):
    moving_avg = np.convolve(x, np.ones(w), 'valid') / w
    padding = np.full_like(np.empty(w), np.nan)
    return np.insert(moving_avg, 0, padding)

min_period = 24*7

moving_average_3_hours = moving_average(df["Close"], 3)
moving_average_12_hours = moving_average(df["Close"], 12)
moving_average_24_hours = moving_average(df["Close"], 24)
moving_average_48_hours = moving_average(df["Close"], 48)
moving_average_1_week = moving_average(df["Close"], 24*7 - 1)

mv_1 = (moving_average_3_hours - moving_average_12_hours) / moving_average_3_hours
mv_2 = (moving_average_12_hours - moving_average_24_hours) / moving_average_12_hours
mv_3 = (moving_average_12_hours - moving_average_24_hours) / moving_average_12_hours
mv_4 = (moving_average_12_hours - moving_average_48_hours) / moving_average_12_hours
mv_5 = (moving_average_12_hours - moving_average_1_week) / moving_average_12_hours

# X = np.stack((moving_average_3_hours, moving_average_12_hours, moving_average_24_hours, moving_average_48_hours, moving_average_1_week), axis=1)
X = np.stack((mv_1, mv_2, mv_3, mv_4, mv_5), axis=1)
X = X[min_period + 1:]
y = y[min_period:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

SVC()

In [33]:
y_train_pred = clf.predict(X_train)
print("Train Accuracy:",metrics.accuracy_score(y_train, y_train_pred))

y_test_pred = clf.predict(X_test)
print("Test Accuracy:",metrics.accuracy_score(y_test, y_test_pred))

Train Accuracy: 0.5308839692259382
Test Accuracy: 0.5140580172138987
